In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.cross_validation import train_test_split
import operator
import xgboost as xgb

In [94]:
properties = pd.read_csv("./data/properties_2016.csv")

In [112]:
train_df = pd.read_csv("./data/train_2016_v2.csv", parse_dates=["transactiondate"])

In [113]:
train_df = pd.merge(train_df, properties, on = "parcelid", how="left")

In [114]:
sample = pd.read_csv('./data/sample_submission.csv')

In [115]:
sample['parcelid'] = sample['ParcelId']
test_df = sample.merge(properties, on='parcelid', how='left')

In [120]:
train_x.head()

,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,1.0,NaN,NaN,2.0,3.0,NaN,4.0,2.0,NaN,NaN,...,NaN,NaN,122754.0,360170.0,2015.0,237416.0,6735.88,NaN,NaN,6.037107e+13
1,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,NaN,NaN,...,NaN,NaN,346458.0,585529.0,2015.0,239071.0,10153.02,NaN,NaN,NaN
2,1.0,NaN,NaN,3.0,2.0,NaN,4.0,3.0,NaN,NaN,...,NaN,NaN,61994.0,119906.0,2015.0,57912.0,11484.48,NaN,NaN,6.037464e+13
3,1.0,NaN,NaN,2.0,2.0,NaN,4.0,2.0,NaN,NaN,...,NaN,NaN,171518.0,244880.0,2015.0,73362.0,3048.74,NaN,NaN,6.037296e+13
4,NaN,NaN,NaN,2.5,4.0,NaN,NaN,2.5,NaN,NaN,...,2.0,NaN,169574.0,434551.0,2015.0,264977.0,5488.96,NaN,NaN,6.059042e+13


In [117]:
train_y = train_df.pop("logerror")
train_x = train_df.drop(['parcelid', 'transactiondate', 'propertyzoningdesc', 'propertycountylandusecode'], axis=1)

test_x = test_df.drop(['parcelid', 'propertyzoningdesc', 'propertycountylandusecode', "ParcelId", "201612", "201610", "201611", "201712", "201711", "201710"], axis=1)

In [121]:
train_x.dtypes[train_x.dtypes == object].index.values

array(['hashottuborspa', 'fireplaceflag', 'taxdelinquencyflag'], dtype=object)

In [122]:
for c in train_x.dtypes[train_x.dtypes == object].index.values:
    train_x.drop(c, axis = 1, inplace = True)
    
for c in test_x.dtypes[test_x.dtypes == object].index.values:
    test_x.drop(c, axis = 1, inplace = True)

In [123]:
def run_xgb(train, label_train, valid = None, label_valid = None):

    # Set our parameters for xgboost
#     params = {}
#     params['objective'] = 'multi:softmax'
#     params['num_class'] = 6
#     params['eta'] = 0.025
#     params['max_depth'] = 6
#     params['silent'] = 1
#     params['min_child_weight'] = 0
#     params['subsample'] = 0.8
#     params['colsample_bytree'] = 0.8
#     params['nthread'] = 13
#     params['gamma'] = 0.1
#     params['max_delta_step'] = 0

    params = {}
    params['objective'] = 'reg:linear'
    params['eval_metric'] = 'mae'
    params['eta'] = 0.05
    params['max_depth'] = 6
    params['silent'] = 1

    d_train = xgb.DMatrix(train, label=label_train)
    
    if valid is not None:
        d_valid = xgb.DMatrix(valid, label=label_valid)
        watchlist = [(d_train, 'train'), (d_valid, 'validation')]
    else:
        watchlist = [(d_train, 'train')]
        
    bst = xgb.train(params, d_train, 200, watchlist, early_stopping_rounds=10, verbose_eval=50)
    
    return bst

In [124]:
train_train, train_valid, train_train_y, train_valid_y = train_test_split(train_x, train_y, test_size=0.2, random_state=4242)

In [125]:
def xgb_explore():
    
#     bst = run_xgb(train_feats, no_of_lanes)
    bst = run_xgb(train_train, train_train_y, train_valid, train_valid_y)

    importance = bst.get_fscore()
    importance = sorted(importance.items(), key=operator.itemgetter(1))

    df_imp = pd.DataFrame(importance, columns=['feature', 'fscore'])
    df_imp['fscore'] = df_imp['fscore']/df_imp['fscore'].sum()

    print df_imp
    return bst

bst = xgb_explore()

[0]	train-mae:0.472925	validation-mae:0.472774
Multiple eval metrics have been passed: 'validation-mae' will be used for early stopping.

Will train until validation-mae hasn't improved in 10 rounds.
[50]	train-mae:0.080782	validation-mae:0.08099
[100]	train-mae:0.067442	validation-mae:0.067916
Stopping. Best iteration:
[125]	train-mae:0.066969	validation-mae:0.067715

                         feature    fscore
0                 regionidcounty  0.000237
1                    poolsizesum  0.000237
2            threequarterbathnbr  0.000237
3                     decktypeid  0.000237
4                   basementsqft  0.000237
5                   pooltypeid10  0.000474
6                           fips  0.000474
7           finishedsquarefeet13  0.000474
8           finishedsquarefeet50  0.001423
9                numberofstories  0.001660
10                   pooltypeid2  0.001897
11             calculatedbathnbr  0.003320
12                       unitcnt  0.003557
13            yardbuilding

In [126]:
d_test = xgb.DMatrix(test_x)
p_test = bst.predict(d_test)

In [127]:
sample.head()

,ParcelId,201610,201611,201612,201710,201711,201712,parcelid
0,10754147,0,0,0,0,0,0,10754147
1,10759547,0,0,0,0,0,0,10759547
2,10843547,0,0,0,0,0,0,10843547
3,10859147,0,0,0,0,0,0,10859147
4,10879947,0,0,0,0,0,0,10879947


In [137]:
sub = pd.read_csv("./data/sample_submission.csv")
for c in sub.columns[sub.columns != 'ParcelId']:
    sub[c] = p_test

In [ ]:
sub.to_csv("xgb_starter.csv", index = False, float_format='%.4f')

In [139]:
len(np.unique(sub))

2985218